# Run Galfit on COSMOS galaxies
- 10/27/2020: Updated to use galfit_helpers.py module

### Load modules

In [1]:
from __future__ import division
import sys
sys.path.insert(0,'/data/emiln/XLSSC122_GalPops/Analysis/Modules')
from GalfitPyWrap import galfitwrap as gf
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import glob
from scipy import ndimage
from astropy import units as u
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy.io import fits, ascii
from astropy.table import Table, hstack, join
import pickle
import fnmatch
import multiprocessing as mp
%matplotlib inline
%load_ext autoreload
%autoreload 2

/data/emiln/miniconda2/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
import galfit_helpers as gfh

### Load COSMOS galaxy catalog

In [11]:
COSMOS_cat_root = '/data/emiln/XLSSC122_GalPops/Data/Products/COSMOS/catalogs/'
COSMOS_cat_filename = COSMOS_cat_root+'cos_df_all_merged.csv'
full_df = gfh.load_COSMOS_galaxy_catalog(COSMOS_cat_filename,magthresh=30, verbose=True)
filtered_df = gfh.load_COSMOS_galaxy_catalog(COSMOS_cat_filename,magthresh=24, verbose=True)


Checking if catalog exists at /data/emiln/XLSSC122_GalPops/Data/COSMOS/Products/catalogs/magthresh30.csv
Catalog already exists, returning DataFrame
Checking if catalog exists at /data/emiln/XLSSC122_GalPops/Data/COSMOS/Products/catalogs/magthresh24.csv
Catalog already exists, returning DataFrame


In [ ]:
filtered_df.co

In [18]:
def fun(a,test=None,test2=None,params=None,**kwargs):
    print(a,test,test2)
    print(params)
params = {'test1':'TEST1','test2':'TEST2','test':'TEST'}
fun('a',params=params,**params)

('a', 'TEST', 'TEST2')
{'test1': 'TEST1', 'test': 'TEST', 'test2': 'TEST2'}


### Set Galfit parameters

In [30]:
fit_df = filtered_df[0:4]
df_name = 'TEST_modular_galfit'
psf_root = '/data/emiln/XLSSC122_GalPops/Data/Products/COSMOS/PSF/'
psf_file = psf_root+'COSMOS/psfJ.fits' # J = F125W, H=F160W
PSFf = 1 
convbox='40 40'
image_width = 200 # 200 for mag<23 objects, 400 for z2 objects
timeout=2*60
verbose=False
width=10
HLRwidth=10 # Cutout width = 2*HLwidth
PA_INIT = 45
AR_INIT = 0.5
# zp = -21.1 # From header
ZP = 26.2303 # From https://archive.stsci.edu/pub/hlsp/candels/cosmos/cos-tot/v1.0/hlsp_candels_hst_cos-tot_readme_v1.0.pdf
MAG_INIT = 21
fitMagnitude = True
neighbourMagThresh=2
DYNMAG = True
sky='Default'
sky_INIT=0.00001
sigma_file = '/sigma_rms_meanexp_cps.fits'
data_file = '/data_cps.fits'
constraint_file = 'galfit_constr.txt' # Matches van der wel 2012 constraints
# constr_file = 'none'
badmask='none'
save_name = None

In [39]:
fit_df.iloc[1]

ID                                                           4
IAU_designation       CANDELS_COSMOS_F160W_J100013.49+021027.5
RA_x                                                   150.056
DEC_x                                                  2.17432
APCOR                                                  1.77497
CFHT_uS_FLUX                                          0.247256
CFHT_uS_FLUXERR                                      0.0207575
CFHT_gS_FLUX                                           0.39028
CFHT_gS_FLUXERR                                      0.0219568
CFHT_rS_FLUX                                          0.447931
CFHT_rS_FLUXERR                                      0.0257099
CFHT_iS_FLUX                                           0.53651
CFHT_iS_FLUXERR                                      0.0338225
CFHT_zS_FLUX                                          0.600002
CFHT_zS_FLUXERR                                      0.0573503
Subaru_B_FLUX                                         0

In [31]:
params = {
    'fit_df':fit_df, # Dataframe with objects to be fit
    'full_df':full_df, # Unfiltered source catalog used for fitting neighbours
    'width':width, # Fitting region width in pixels
    'HLRwidth':HLRwidth, # Fitting region width in # of HLR
    'sigma_file':sigma_file, # Filename of sigma maps for sources
    'data_file':data_file, # Filename for data cutouts for each source
    'PSF_file':psf_file, # File_name of PSF to be used
    'usePSF':True, # Use PSF in fitting?
    'timeout':timeout, # Max runtime per object in seconds
    'PSFf':PSFf, # Fine sampling factor
    'verbose':verbose, # Verbose mode
    'PA_INIT':PA_INIT, # Initial position angle
    'AR_INIT':AR_INIT, # Initial axis ratio
    'MAG_INIT':MAG_INIT, # Initial magnitude
    'convbox':convbox, # Region to convolve PSF with in pixels (i.e. '100 100')
    'ZP':ZP, # Zeropoint 
    'constraint_file':constraint_file, # Galfit constraint filename
    'image_width':image_width, # Size of data+sigma images being used (200 for COSMOS cutouts)
    'useDYNMAG':DYNMAG, # Initialize magnitudes from catalog?
    'badmask':badmask, # filename for bad pixel mask
    'fitMagnitude':fitMagnitude, # Fit magnitudes?
    'sky':sky, # Sky fitting mode for galfit (i.e. 'default')
    'sky_INIT':sky_INIT, # Initial sky level
    'neighbourMagThresh':neighbourMagThresh, # Additional magnitude threshhold to fit neighbours (i.e. 3 -> only neighbours with mag < source_mag+3 are fit)
    'df_name': df_name, # Descriptive name of catalog being fit
    'save_name':save_name # Filename to save results to, overrides default
}

In [36]:
gfh.run_galfit_parallel(params,**params) # Feeds in unwrapped params as well as a copy to be passed as kwargs to sub functions

SAVE_NAME:TEST_modular_galfit_data_cps_sigma_rms_meanexp_cps_w10HLR_/data/emiln/XLSSC122_GalPops/Data/Products/COSMOS/PSF/COSMOS/psfJ_2min_CONV40_CONSTR_DYNMAG
Initializing 1 with F125W Kron magnitude: 21.16
ID 1
RA: 150.0568177
DEC: 2.173421
Initial X: 10968.1486249
Initial Y: 7547.44464024
Cutout X: 200
Cutout Y: 200
Cutout width: 200
Cutoutwidth (pixels) for ID 1 : 116
Cutoutwidth (arcsec) for ID 1 : 6.96
Bounds: [142, 258, 142, 258]
4 NEIGHBOURS FOUND
Adding additional model components for neighbours...
NEIGHBOUR initialized with F125W Kron magnitude: 22.51
NEIGHBOUR initialized with F125W Kron magnitude: 29.21
Neighbour mag too faint, not being fit
NEIGHBOUR initialized with F125W Kron magnitude: 23.76
Neighbour mag too faint, not being fit
NEIGHBOUR initialized with F125W Kron magnitude: 24.76
Neighbour mag too faint, not being fit
Using PSF


IOError: [Errno 2] No such file or directory: '/data/emiln/XLSSC122_GalPops/Data/Products/COSMOS/galfit_results/1/input.feedme'

In [ ]:
tdf = cos_df_highz[4:8]

df_name = 'cos_highz_mag24'

print len(tdf)

print 'FINAL ID:', max(tdf['NUMBER'])

pool = mp.Pool(4) 

psf_root = '/data/emiln/XLSSC122_GalPops/Data/Products/COSMOS/PSF/'
psf_file = psf_root+'COSMOS/psfJ.fits' # J = F125W, H=F160W
psf_name = 'cosmosPSFJ'

PSFf = 1 # Fine sampling factor of PSF (integer). COSMOS image is 0.06''/pixel. If PSF is 0.03''/pixel -> PSFf = 2
convbox='40 40'
cutout_width = 400 # 200 for mag<23 objects, 400 for z2 objects
# convbox='200 200'
# timeout=2*300 # maximum fit run time, test 30min runtime, if running on 8 threads, 4 processes will idle --> set timeout to 2* normal
timeout=5*60
verb=False

w=10
HLwidth=10 # Cutout width = 2*HLwidth
# HLwidth=False

PA_INIT = 45
AR_INIT = 0.5
# zp = -21.1 # From header
zp = 26.2303 # From https://archive.stsci.edu/pub/hlsp/candels/cosmos/cos-tot/v1.0/hlsp_candels_hst_cos-tot_readme_v1.0.pdf
MAG_INIT = 21
MAGFIT = True
mag_thresh=2
DYNMAG = True
sky='Default' # {'Default','None'} : Default = {0: 'sky', 1: str(skyINIT)+' 1', 2: '0 0', 3: '0 0', 'Z': 0, 'Comment': 'StandardSky'}
# sky = 'None'
skyINIT=0.00001

# sigma_file = '/sigma_meanexp_cutout.fits'
sigma_file = '/sigma_rms_meanexp_cps.fits'
# sigma_file = '/sigma_meanexp_counts.fits'
# sigma_file = '/sigma_meanexp_counts_ADU_exp_adjusted.fits' #exposure header adjusted to 1900s --> https://archive.stsci.edu/pub/hlsp/candels/cosmos/cos-tot/v1.0/hlsp_candels_hst_cos-tot_readme_v1.0.pdf

# cutout_file = '/cutout.fits'
cutout_file = '/data_cps.fits'

constr_file = 'galfit_constr.txt' # Matches van der wel 2012 constraints
# constr_file = 'none'
test = True

badmask='none'
# badmask='/20p_mask.fits'

save_name = df_name+'_'+cutout_file[1:-5]+'_'+sigma_file[1:-5]+'_w'+\
            [str(HLwidth)+'HLR' if HLwidth!=False else str(w)][0]+'_'+psf_name+'_'+\
            str(int(timeout/60))+'min_CONV'+convbox.split(' ')[0]+\
            ['_CONSTR' if constr_file !='none' else ''][0]+['_DYNMAG' if DYNMAG else ''][0]+\
            ['_TEST' if test else ''][0]

print 'SAVE_NAME:', save_name


# results = [pool.apply_async(run_galfit_cosmos_parallel, args=([r])) for idx, r in tdf.iterrows()]

results = [pool.apply_async(run_galfit_parallel, args=([r]), kwds={'OG_df':cos_df_OG,'width':w,\
            'sigma_file':sigma_file,'cutout_file':cutout_file,\
            'save_name':save_name,'psf_file':psf_file,\
            'use_psf':True,'timeout':timeout,'PSFf':PSFf,'verb':verb,'HLwidth':HLwidth,\
             'PA_INIT':PA_INIT, 'AR_INIT':AR_INIT, 'MAG_INIT':MAG_INIT, 'convbox':convbox,\
            'zp':zp,'constr':constr_file,'cutout_width':cutout_width,'DYNMAG':DYNMAG,\
             'badmask':badmask,'MAGFIT':MAGFIT, 'sky':sky,'skyINIT':skyINIT, 'mag_thresh':mag_thresh}) for idx, r in tdf.iterrows()]

output = [p.get() for p in results]
pool.close()
print output

new_df = pd.DataFrame(output,columns=['ID','ra','dec','re','ar','n','mag','sky','chi2nu','ErrorValue']) 
# ErrorValue of 124 = process timeout, 1 = GALFIT exception
results_root = '/data/emiln/XLSSC122_GalPops/Analysis/COSMOS/Results/'
new_df.to_csv(results_root+save_name+'.csv',index=False)
new_df

### Run Galfit

### Validate results

### Save results